# TensorFlow Basic Neural Network (with Only Tensor Operations)

Replicated from [Aymeric Damien's TF `neural_network_raw` Example](https://github.com/aymericdamien/TensorFlow-Examples/blob/master/notebooks/3_NeuralNetworks/neural_network_raw.ipynb)

Basic multi-layer perceptron neural network, without any pre-built layer or cell. All just matrix/tensor operations.

### Load packages and MNIST data set

In [44]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/tmp/data/', one_hot=True)

import tensorflow as tf

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


### Hyper parameters for learning

In [45]:
learning_rate = 0.1
num_steps = 5000
batch_size = 128
display_step = 1000

In [46]:
n_hidden_1 = 256  # number of neurons
n_hidden_2 = 256
num_input = 28**2
num_classes = 10

### TF graph elements

Outputs `X` and `Y` are placeholder tensors each with 1 unspecified dimention. (It's the dim representing observations, we know this dim exists, but don't know the exact number.)

In [47]:
X = tf.placeholder(dtype=tf.float32, shape=[None, num_input])
Y = tf.placeholder(dtype=tf.float32, shape=[None, num_classes])

Transformations between layers are represented as matrices (2D tensors).

In [48]:
weights = {
    'h1': tf.Variable(tf.random_normal(shape=[num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal(shape=[n_hidden_1, n_hidden_2])),
    'out':tf.Variable(tf.random_normal(shape=[n_hidden_2, num_classes]))
}
biases = {
    'h1': tf.Variable(tf.random_normal(shape=[n_hidden_1])),
    'h2': tf.Variable(tf.random_normal(shape=[n_hidden_2])),
    'out':tf.Variable(tf.random_normal(shape=[num_classes]))
}

The neural network is essentially a chain of operations.

In [49]:
def neural_net(x):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['h1'])
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['h2'])
    out_layer = tf.add(tf.matmul(layer_2, weights['out']), biases['out'])
    return out_layer

In [50]:
logits = neural_net(X)

Loss function is defined as cross entropy between predicted probability of all labels (0-9) and true labels.

In [51]:
loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
        logits=logits, labels=Y
    )
)

Adam Optimizer is probably the best optimizer out there.

In [52]:
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

In [53]:
train = optimizer.minimize(loss)

In [54]:
pred = tf.argmax(logits, axis=1)
true = tf.argmax(Y, axis=1)
correct = tf.equal(pred, true)
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))

In [55]:
init = tf.global_variables_initializer()    # init is a <tf.Operation> object

### Train the network

Train the network (optimize the `weights` and `biases`) in a `tf.Session()`

In [56]:
with tf.Session() as sess:
    sess.run(init)
    
    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size=batch_size)
        
        sess.run(train, feed_dict={X: batch_x, Y: batch_y})
        
        if step % display_step ==0 or step ==1:
            los, acc = sess.run([loss, accuracy], feed_dict={X: batch_x, Y: batch_y})
            print('Step ' + str(step) + \
                  ', Minibatch Loss= ' + '{:.4f}'.format(los) + \
                  ', Training Accuracy=' + '{:0.3f}'.format(acc) )
    
    print('Training completed.')
    print('Accuracy on test set:', sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels}))

Step 1, Minibatch Loss= 10877.0234, Training Accuracy=0.305
Step 1000, Minibatch Loss= 74.8363, Training Accuracy=0.859
Step 2000, Minibatch Loss= 44.2013, Training Accuracy=0.859
Step 3000, Minibatch Loss= 63.5150, Training Accuracy=0.859
Step 4000, Minibatch Loss= 364.4888, Training Accuracy=0.859
Step 5000, Minibatch Loss= 56.4700, Training Accuracy=0.930
Training completed.
Accuracy on test set: 0.87


In [61]:
a = tf.summary.FileWriter(logdir='tflog')

In [ ]:
a.add_summary()